# General input prep

### Fetch structure entry from OPTIMADE

In [ ]:
from optimade.client import OptimadeClient

url = "https://aiida.materialscloud.org/mc3d/optimade"
query = 'elements HAS "Si" AND nsites < 2'
client = OptimadeClient(url)

In [ ]:
%%capture

_ = client.get(query)

In [ ]:
results = client.all_results["structures"][query][url]

In [ ]:
structure_entry = results.data[0]

In [ ]:
from pathlib import Path

relax_engine = {
    "code": {
        # TODO handle cases where installing a new code could be sufficient
        "identifier": "",  # AiiDA code identifier
        "executed_by": {
            "name": "python",
            "version": "3.10.16",
            "availability": {
                "package_manager": {
                    "name": "conda",
                    "version": "24.7.1",
                },
                "package": "python",
            },
        },
        "executed_on": {
            "hostname": "localhost",
            "architecture": "x86_64",
            "os": {
                "name": "Linux",
                "metadata": {
                    "distribution": {
                        "name": "Ubuntu",
                        "version": "24.04.2",
                    }
                },
            },
        },
        "executable_path": (Path(".") / "hello_world.py").absolute().as_posix(),
    },
    "options": {
        "resources": {
            "num_machines": 1,
        },
        "max_wallclock_seconds": 3600,
    },
}

inputs = {
    "structure": structure_entry,
    "engines": {
        "relax": relax_engine,
    },
    "protocol": "fast",
    "relax_type": "positions",
    "threshold_forces": 0.01,
    "threshold_stress": 0.1,
    "reference_process": None,  # Optional AiiDA process UUID
}

# Internal conversion to AiiDA

In [ ]:
def pretty(d: dict, indent: int = 4, level: int = 0):
    for key, value in d.items():
        print(" " * indent * level + str(key))
        if isinstance(value, dict):
            pretty(value, indent, level + 1)
        else:
            print(" " * indent * (level + 1) + str(value))

In [ ]:
from aiida import orm, common, load_profile

load_profile();

In [ ]:
from common_workflow_schemas.schemas.relax import RelaxInputsModel

inputs_model = RelaxInputsModel(**inputs)

In [ ]:
# inputs_model.model_dump()

In [ ]:
# pretty(inputs_model.model_json_schema(), indent=2)

In [ ]:
from optimade.adapters import Structure

structure_entry = inputs_model.structure.model_dump()
structure = Structure(structure_entry).convert(format="aiida_structuredata")

In [ ]:
import typing as t

def load_node(identifier: t.Union[str, int]) -> t.Optional[orm.Node]:
    try:
        return orm.load_node(identifier)
    except common.exceptions.NotExistent:
        return None

In [ ]:
reference_process = load_node(inputs_model.reference_process)

In [ ]:
def load_code(engine: str):
    return orm.load_code(inputs_model.engines[engine].code.identifier)

In [ ]:
acwf_input = {
    "structure": structure,
    "engines": {
        engine: {
            "code": load_code(engine),
            "options": inputs_model.engines[engine].options,
        }
        for engine in inputs_model.engines
    },
    "protocol": inputs_model.protocol,
    "relax_type": inputs_model.relax_type,
    "threshold_forces": orm.Float(inputs_model.threshold_forces),
    "threshold_stress": orm.Float(inputs_model.threshold_stress),
    "electronic_type": orm.Str(inputs_model.electronic_type)
    if inputs_model.electronic_type
    else None,
    "spin_type": orm.Str(inputs_model.spin_type) if inputs_model.spin_type else None,
    "reference_workchain": reference_process,
}

In [ ]:
pretty(acwf_input, indent=2)
